In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
num_steps=10000
batch_size=128
learning_rate = 0.0002
# G , D - DNN (256개)
image_dim = 784
gen_hidden_dim = 256
disc_hidden_dim = 256
noise_dim = 100

In [6]:
# 사비에르 구현

In [7]:
def he_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.)) 

weights = {
    'gen_hidden1': tf.Variable(he_init([noise_dim, gen_hidden_dim])),
    'gen_out' : tf.Variable(he_init([gen_hidden_dim, image_dim])),
    'disc_hidden1' : tf.Variable(he_init([image_dim, disc_hidden_dim])),
    'disc_out' : tf.Variable(he_init([disc_hidden_dim,1]))
}

biases = {
    'gen_hidden1' : tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_out': tf.Variable(tf.zeros([image_dim])),
    'disc_hidden1' : tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc_out' : tf.Variable(tf.zeros([1]))
}

In [8]:
def generator(x):
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['gen_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    output_layer = tf.matmul(hidden_layer, weights['gen_out'])
    output_layer = tf.add(output_layer, biases['gen_out'])
    output_layer = tf.nn.sigmoid(output_layer)
    return output_layer

def discriminator(x):
    hidden_layer = tf.matmul(x, weights['disc_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    output_layer = tf.matmul(hidden_layer, weights['disc_out'])
    output_layer = tf.add(output_layer, biases['disc_out'])
    output_layer = tf.nn.sigmoid(output_layer)
    return output_layer    

In [9]:
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
disc_input = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input')

gen_sample = generator(gen_input)
disc_real = discriminator(disc_input) # Real 데이터를 받는 애랑
disc_fake = discriminator(gen_sample) # Facke 이미지를 받는 애 두개

gen_loss = -tf.reduce_mean(tf.log(disc_fake))
# 밑에 식이 log(D(x))+log(1-D(G(x)))
disc_loss = -tf.reduce_mean(tf.log(disc_real) + tf.log(1.- disc_fake))

optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

gen_vars = [weights['gen_hidden1'], weights['gen_out'],
           biases['gen_hidden1'], biases['gen_out']]
disc_vars = [weights['disc_hidden1'], weights['disc_out'],
           biases['disc_hidden1'], biases['disc_out']]

train_gen = optimizer_gen.minimize(gen_loss, var_list = gen_vars)
train_disc = optimizer_disc.minimize(disc_loss, var_list = disc_vars)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(num_steps):
    batch_xs, _ = mnist.train.next_batch(batch_size) # disc의 인풋
    z = np.random.uniform(-1.,1.,size=[batch_size, noise_dim]) # gen 인풋
    _, _, gl, dl = sess.run([train_gen,train_disc,gen_loss, disc_loss],
                        feed_dict={disc_input:batch_xs, gen_input:z})
    if i % 100 == 0:
        print("Step:",i,"G Loss:",gl,"D Loss:",dl)

Step: 0 G Loss: 0.28461903 D Loss: 1.9181436
Step: 100 G Loss: 4.3973317 D Loss: 0.047020227
Step: 200 G Loss: 3.4672449 D Loss: 0.14827183
Step: 300 G Loss: 2.5136147 D Loss: 0.34276146
Step: 400 G Loss: 2.8394322 D Loss: 0.24390507
Step: 500 G Loss: 3.0885825 D Loss: 0.12815924
Step: 600 G Loss: 3.2595859 D Loss: 0.13474798
Step: 700 G Loss: 3.4005065 D Loss: 0.08104004
Step: 800 G Loss: 3.5552144 D Loss: 0.078812055
Step: 900 G Loss: 3.6366167 D Loss: 0.06815943
Step: 1000 G Loss: 3.6196246 D Loss: 0.09625015
Step: 1100 G Loss: 3.5145905 D Loss: 0.06541948
Step: 1200 G Loss: 3.52983 D Loss: 0.0768328
Step: 1300 G Loss: 3.7536216 D Loss: 0.09620807
Step: 1400 G Loss: 3.868499 D Loss: 0.077326566
Step: 1500 G Loss: 4.147256 D Loss: 0.0561004
Step: 1600 G Loss: 4.3674355 D Loss: 0.051446926
Step: 1700 G Loss: 4.011893 D Loss: 0.06920449
Step: 1800 G Loss: 3.9665887 D Loss: 0.104530305
Step: 1900 G Loss: 4.5099363 D Loss: 0.04063089
Step: 2000 G Loss: 4.8489175 D Loss: 0.03483531
Step: 